In [16]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [5]:
import json
import pandas as pd
import os
import numpy as np

In [6]:
data = {}
clean_data = {}

for f in os.listdir('/home/katya/start/HIS3InterspeciesEpistasis/Data/'):
    if 'csv' in f:
        data[f[:-16]] = pd.DataFrame.from_csv('/home/katya/start/HIS3InterspeciesEpistasis/Data/' + f, sep='\t')
        median = np.median(data[f[:-16]].index.str.len())
        
        clean_data[f[:-16]] = data[f[:-16]][(data[f[:-16]].nonsense == 0) & (data[f[:-16]].middle == 1) & \
                                            (data[f[:-16]].nat_lib == 1) & (data[f[:-16]].stop == 0) & \
                                            (data[f[:-16]].index.str.len() == median)]

In [8]:
source = ['23','23','23','24','24','40','40','41','41','25','25','26','26','27','27','28','28','34','34','35','35','36','36','37','37','38','38','39','39','29','29','30','30','31','31','32','32','33','33']
target = ['Ylip','Anid','24','40','25','Sjap','41','Soct','Spom','Ncra','26','Agos','27','Cgla','28','29','34','Cgui','35','36','37','Clus','Dhan','38','39','Calb','Ctro','Cpar','Lelo','30','32','Scas','31','Scer','Sbay','Klac','33','Kwal','Sklu']

nodes = list(source)
nodes.extend(target)
nodes = list(set(nodes))

In [83]:
chunk = 'S11 '

data = {}
data['nodes'] = []
for node in nodes:

        if node.isdigit():
            if len(clean_data[chunk][clean_data[chunk]['dist_node_'+node]==0])!=0:
                fitness = float(clean_data[chunk][clean_data[chunk]['dist_node_'+node]==0].s)
                
            else:
                fitness = 0
        else:
            if len(clean_data[chunk][clean_data[chunk]['dist_'+node]==0])!=0:
                fitness = float(clean_data[chunk][clean_data[chunk]['dist_'+node]==0].s)
                
            else:
                fitness = 0
                
        data['nodes'].append({'id':node, 'fitness':fitness, 'group':0})

In [84]:
data['links'] = []

for i in range(len(source)):
    data['links'].append({'source':source[i], 'target':target[i], 'group':0})

In [85]:
tempNodes = []

for node in nodes:
    if node.isdigit():
        nodeList = clean_data[chunk][clean_data[chunk]['dist_node_'+node]==1].index
        for sq in nodeList:
            if sq+node+'1' not in tempNodes:
                tempNodes.append(sq+node+'1')
                data['nodes'].append({'id':sq+node+'1', 'fitness':clean_data[chunk].ix[sq].s, 'group':1})
            data['links'].append({'source':node, 'target':sq+node+'1', 'group':1})
            
    else:
        nodeList = clean_data[chunk][clean_data[chunk]['dist_'+node]==1].index
        for sq in nodeList:
            if sq+node+'1' not in tempNodes:
                tempNodes.append(sq+node+'1')
                data['nodes'].append({'id':sq+node+'1', 'fitness':clean_data[chunk].ix[sq].s, 'group':1})
            data['links'].append({'source':node, 'target':sq+node+'1', 'group':1})

In [86]:
with open('/home/katya/start/HIS3InterspeciesEpistasis/Analysis/Katya/Fig3/test.json', 'w+') as outfile:  
    json.dump(data, outfile)